# Database to Google Sheets Converter

By Kenneth Burchfiel

Released under the MIT license

This program demonstrates how to convert a local SQLite database into a Google Sheets workbook.

In [1]:
import time
start_time = time.time() # Allows the program's runtime to be measured
import pandas as pd
import sqlalchemy
import gspread
from gspread_dataframe import set_with_dataframe

# Prerequisites

Before you can apply this code to your own projects, you'll need to perform some setup tasks. These tasks are based on the tasks for my Google Slides Image Uploader
script, available at this repository: https://github.com/kburchfiel/google_slides_image_uploader/

## Step 1:
Open a Google Cloud Platform project. I used the Google Cloud Console to accomplish this step. For instructions, go to https://cloud.google.com/resource-manager/docs/creating-managing-projects#console. 

NOTE: You may incur expenses when using the Google Cloud platform.

## Step 2:
Enable the Google Sheets API for your project. To do so, enter 'Sheets API' within the search box near the top of the Google Cloud Platform window. Click on the 'Google Sheets API' result and then select the blue 'Enable' button. 

## Step 3:
Create a Google service account. You can do so by following the steps shown in Google's [Create service accounts](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) documentation page. (Although this page instructs you to "enable the IAM API," I didn't need to do so in order for the following steps to work, but it's possible that this API had been enabled beforehand for my Cloud Console project.)

## Step 4:
Create a key in JSON format for this service account, then download it to your computer (as a .json file) and store it in a safe location. See https://cloud.google.com/iam/docs/creating-managing-service-account-keys 

## Step 5:
Grant this service account Editor access to each Google Sheet to which you will need to connect. You can grant it access by clicking the 'Share' button within the presentation and then entering the service account's email within the box that appears. The address of this service account can be found within the 'Service account details' page of your service account within the Google Cloud platform.




First, I'll access my service account key by opening the path to my keys folder, then retrieving the account key within that folder.

In [2]:
with open('../key_paths/path_to_keys_folder.txt') as file:
    path_to_keys_folder = file.readline() 

In [3]:
gc = gspread.service_account(
    path_to_keys_folder+'/kburchfiel-public-programs-service-account-key.json') 
    # Based on https://docs.gspread.org/en/latest/oauth2.html
# This is the path to my downloaded Google Service Account key, which is 
# necessary for connecting to Google Sheets documents from your computer.

The following code block connects to my SQLite database.

<span style="color:red"> **Note:** Due to its size, I could not upload sqlite_database.db, the SQLite database that this script connects to, directly to GitHub. You can instead find it via Google Drive at the following link:</span> 

https://drive.google.com/file/d/1THyBZYXXT4le6zQz2SBhQlIjMk2MjhYz/view?usp=sharing

Once you have downloaded sqlite_database.db, update the file path below to match the location of your copy of the database on your computer.

In [4]:
sqlalchemy_sqlite_engine = sqlalchemy.create_engine(
'sqlite:///'+'C:/Users/kburc/D1V1/Documents/!Dell64docs/Programming/py/\
kjb3_programs/python_database_utilities/data/sqlite_database.db') 
# Based on:
#  https://docs.sqlalchemy.org/en/13/dialects/sqlite.html#connect-strings



Next, I'll connect to my Google Sheets file. You won't be able to connect to this file because you don't have a copy of my service account key; however, I recommend creating your own service key and Google Sheets file so that you can try out this method on your own.

In [5]:
db_workbook = gc.open_by_key('1iJs9ldwsysMP-yTkM447Qyz5do1KACAUCIIivuOB6yM')
# The key was retrieved from the document's URL:
# https://docs.google.com/spreadsheets/d/1iJs9ldwsysMP-yTkM447Qyz5do1KACAUCIIivuOB6yM/edit


The following code block creates a list of all tables within the SQLite database.

In [6]:
table_query = sqlalchemy_sqlite_engine.execute("Select name from sqlite_schema \
where type = 'table'") 
# This method for checking whether a certain table already exists within the 
# database comes from: 
# https://www.kite.com/python/answers/how-to-list-tables-using-sqlite3-in-python
table_tuple_list = table_query.fetchall() # Returns a set of tuples
table_list = [query[0] for query in table_tuple_list] # List comprehension 
# extracts only the first part of each tuple stored in table_list.

print(table_list)

['flights', 'photos', 'steps', 'music']


The following code block retrieves all worksheet titles within the Googe Sheets workbook. It then compares this list with the list of tables in the SQLite database. If a given SQLite table name is not present in the list of worksheet titles, a new worksheet will be added with that table name.

In [7]:
worksheet_titles = []
worksheet_list = db_workbook.worksheets()
for worksheet in worksheet_list:
    worksheet_titles.append(worksheet.title)
# worksheet.title comes from Tanaike at
# https://stackoverflow.com/a/59887962/13097194

for table in table_list:
    if table not in worksheet_titles:
        db_workbook.add_worksheet(table, rows=1, cols = 1) 
# The row and column count will be expanded to match the size of the table 
# during the import process.

Next, each SQLite table will be read into a DataFrame (df_table). 

In [8]:
for i in range(len(table_list)):
# The total cell count of each table is checked beforehand due to Google Sheets' 
# limit of 5,000,000 cells within a given table. If this limit is exceeded,
# that table will not be uploaded to Google Sheets. 
# Checking the cell count of a given table using the following method appears 
# to be much faster than first reading the entire table into a DataFrame, then 
# checking the length of that table. Therefore, these checks are carried
# out before each DataFrame is imported.
    row_count = row_count = sqlalchemy_sqlite_engine.execute(
    "Select count(*) from "+table_list[i]).fetchone()[0] 
    # fetchone() returns a tuple, the first part of which contains 
    # the row count,
    # hence the inclusion of [0] after fetchone().
    column_count = len(pd.read_sql("Select * from "+table_list[i]+" limit 1", 
    con = sqlalchemy_sqlite_engine).columns)
    cell_count = row_count * column_count
    # print(cell_count)
    if cell_count > 5000000:
        print("Google Sheets supports only 5,000,000 cells within a single \
worksheet. Therefore, the table '"+table_list[i]+"' was not added into \
Google Sheets, as its cell count is "+str(cell_count)+".") 
    else:
        df_table = pd.read_sql("Select * from "+table_list[i], 
        con = sqlalchemy_sqlite_engine, index_col = 'index')
        table_worksheet = db_workbook.worksheet(table_list[i]) # Selects the 
        # worksheet by title 
        # (see https://docs.gspread.org/en/latest/user-guide.html) 
        # An earlier code
        # block verified that each table in the SQLite database has 
        # a corresponding
        # Google Sheet worksheet with the same name.
        table_worksheet.clear()
        set_with_dataframe(table_worksheet, df_table, include_index = True)


Google Sheets supports only 5,000,000 cells within a single worksheet. Therefore, the table 'flights' was not added into Google Sheets, as its cell count is 30383199.


Finally, the next code block imports each Google Sheet back into a DataFrame in order to ensure that the upload was successful.

In [9]:
df_output_list = []

for i in range(len(table_list)):
    table_worksheet = db_workbook.worksheet(table_list[i])
    df_sheet = pd.DataFrame(table_worksheet.get_all_records())
    if 'index' in df_sheet.columns:
        df_sheet.set_index('index',inplace=True)
    df_output_list.append(df_sheet)

In [10]:
df_output_list[0] # Returns an empty DataFrame because the 'flights' table was
# too large to upload to Google Drive.

""


In [11]:
df_output_list[1]

,file_name,size,created_date,modified_date,gcs_url
index,,,,,
0,10086848403_b33a695758_o.jpg,1027202,Sat Nov 13 11:34:49 2021,Mon Sep 13 20:44:36 2021,https://storage.googleapis.com/kb_sample_datab...
1,10171744985_76f8973d6b_o.jpg,160300,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:42:14 2021,https://storage.googleapis.com/kb_sample_datab...
2,10822424126_91be9abb6d_o.jpg,402898,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:42:25 2021,https://storage.googleapis.com/kb_sample_datab...
3,10843413524_82caa8b0f8_o.jpg,7405423,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:32:56 2021,https://storage.googleapis.com/kb_sample_datab...
4,11309341065_6fcfbee752_o.jpg,466609,Sat Nov 13 11:34:49 2021,Mon Sep 13 20:38:00 2021,https://storage.googleapis.com/kb_sample_datab...
...,...,...,...,...,...
128,S69-34316~orig.jpg,1339165,Sat Nov 13 11:34:51 2021,Mon Sep 13 01:45:54 2021,https://storage.googleapis.com/kb_sample_datab...
129,S71-41357~orig.jpg,2103357,Sat Nov 13 11:34:51 2021,Mon Sep 13 20:48:45 2021,https://storage.googleapis.com/kb_sample_datab...
130,S71-41759~orig.jpg,1470210,Sat Nov 13 11:34:51 2021,Mon Sep 13 20:07:29 2021,https://storage.googleapis.com/kb_sample_datab...


In [12]:
df_output_list[2]

,dateTime,value
index,,
0,2020-01-20 19:05:00,0
1,2020-01-20 19:38:00,0
2,2020-01-20 19:39:00,0
3,2020-01-20 19:40:00,0
4,2020-01-20 19:41:00,61
...,...,...
529251,2021-09-16 18:26:00,0
529252,2021-09-16 18:27:00,0
529253,2021-09-16 18:28:00,0


In [13]:
df_output_list[3]

,music_file_name,music_size,music_created_date,music_modified_date,music_gcs_url
index,,,,,
0,sample_0.mp3,94391,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:32:27 2021,https://storage.googleapis.com/kb_sample_datab...
1,sample_1.mp3,114244,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:40:16 2021,https://storage.googleapis.com/kb_sample_datab...
2,sample_10.mp3,73493,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:14 2021,https://storage.googleapis.com/kb_sample_datab...
3,sample_11.mp3,94391,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:18 2021,https://storage.googleapis.com/kb_sample_datab...
4,sample_12.mp3,64088,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:22 2021,https://storage.googleapis.com/kb_sample_datab...
...,...,...,...,...,...
59,sample_62.mp3,73493,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:38:15 2021,https://storage.googleapis.com/kb_sample_datab...
60,sample_63.mp3,103795,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:38:19 2021,https://storage.googleapis.com/kb_sample_datab...
61,sample_7.mp3,64088,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:03 2021,https://storage.googleapis.com/kb_sample_datab...


That's it for this program! I hope you'll find this code useful for your own database export projects.

In [14]:
end_time = time.time()
run_time = end_time - start_time
run_minutes = run_time // 60
run_seconds = run_time % 60
print("Completed run at",time.ctime(end_time),"(local time)")
print("Total run time:",'{:.2f}'.format(run_time),"second(s) ("+str(
run_minutes),"minute(s) and",'{:.2f}'.format(run_seconds),"second(s))") 
# Only meaningful when the program is run nonstop from start to finish

Completed run at Wed Sep 13 19:59:06 2023 (local time)
Total run time: 72.01 second(s) (1.0 minute(s) and 12.01 second(s))
